In [1]:
# 데이터분석 관련 모듈 로딩
import pandas as pd
import numpy as np


# 모델링 관련 모듈 로딩
# Pytorch 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchinfo import summary
from torchmetrics.classification import BinaryF1Score
import torch.optim.lr_scheduler as lr_scheduler


# scikit-learn 관련
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 사용자 생성 모듈
import funcMake

# 그 외 기타 모듈 로딩
import os
import pickle

In [2]:
# 데이터 파일 및 경로 지정
FILE = 'data2.xlsx'
PATH = './Data/'
funcMake.checkPath(PATH+FILE)

경로있음


In [3]:
# 데이터 확인
dataDF = pd.read_excel(PATH+FILE)

In [4]:
dataDF.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
enc_list=[]
# 어뷰징여부 컬럼 의심됨(0), 정상가(1) 인코딩
for data in dataDF['체크']:
    if data == '의심됨': enc_list.append(0)
    else: enc_list.append(1)

In [6]:
dataDF['어뷰징여부(0, 1)'] = enc_list

In [7]:
dataDF.head()

,Unnamed: 0.1,상점,가격,상품명,시간,체크,"어뷰징여부(0, 1)"
0,0,올리브앤파인,7550,락토핏 골드 2g x 50포,10월 16일 00시 20분,의심됨,0
1,1,더건강한몰,7560,락토핏 골드 2g x 50포,10월 16일 00시 20분,의심됨,0
2,2,NaN,8420,락토핏 골드 2g x 50포,10월 16일 00시 20분,정상가,1
3,3,A 에이든,8420,락토핏 골드 2g x 50포,10월 16일 00시 20분,정상가,1
4,4,홈쇼핑앤샵,8620,락토핏 골드 2g x 50포,10월 16일 00시 20분,정상가,1


In [26]:
dataDF['체크'].value_counts() # 의심됨: 0, 정상가: 1

체크
정상가    66384
의심됨    45626
Name: count, dtype: int64

In [25]:
dataDF['어뷰징여부(0, 1)'].sum()

66384

In [8]:
# 중간 저장 부분
checkpoint='point.xlsx'
if not os.path.exists(PATH+checkpoint):
    dataDF.to_excel(PATH+checkpoint)
else:
    pass

In [9]:
# 라벨/정수 인코더 인스턴스 생성
lencoder = LabelEncoder()

In [10]:
product_enc=lencoder.fit_transform(dataDF['상품명'])

In [11]:
dataDF['상품(encoding)']=product_enc

In [12]:
dataDF.head()

,Unnamed: 0.1,상점,가격,상품명,시간,체크,"어뷰징여부(0, 1)",상품(encoding)
0,0,올리브앤파인,7550,락토핏 골드 2g x 50포,10월 16일 00시 20분,의심됨,0,41
1,1,더건강한몰,7560,락토핏 골드 2g x 50포,10월 16일 00시 20분,의심됨,0,41
2,2,NaN,8420,락토핏 골드 2g x 50포,10월 16일 00시 20분,정상가,1,41
3,3,A 에이든,8420,락토핏 골드 2g x 50포,10월 16일 00시 20분,정상가,1,41
4,4,홈쇼핑앤샵,8620,락토핏 골드 2g x 50포,10월 16일 00시 20분,정상가,1,41


In [13]:
newDF=dataDF[['상품명','상품(encoding)','어뷰징여부(0, 1)','시간','가격']]
newDF

,상품명,상품(encoding),"어뷰징여부(0, 1)",시간,가격
0,락토핏 골드 2g x 50포,41,0,10월 16일 00시 20분,7550
1,락토핏 골드 2g x 50포,41,0,10월 16일 00시 20분,7560
2,락토핏 골드 2g x 50포,41,1,10월 16일 00시 20분,8420
3,락토핏 골드 2g x 50포,41,1,10월 16일 00시 20분,8420
4,락토핏 골드 2g x 50포,41,1,10월 16일 00시 20분,8620
...,...,...,...,...,...
112005,세노비스 트리플러스 50+ 1305mg x 90캡슐,60,1,10월 27일 07시 38분,33920
112006,세노비스 트리플러스 50+ 1305mg x 90캡슐,60,1,10월 27일 07시 38분,34010
112007,세노비스 트리플러스 50+ 1305mg x 90캡슐,60,1,10월 27일 07시 38분,35000
112008,세노비스 트리플러스 50+ 1305mg x 90캡슐,60,1,10월 27일 07시 38분,35430


In [14]:
# 중간 저장
# checkpoint2='point2.xlsx'
# if not os.path.exists(PATH+checkpoint2):
#     newDF.to_excel(PATH+checkpoint2)
# else:
#     pass

- 라벨인코딩 여부 검증

In [20]:
def checkEncoding(label):
    check_freg={}
    for data in dataDF[label]:
        if data not in check_freg:
            check_freg[data] = 1
        else:
            check_freg[data] += 1
    return check_freg

In [16]:
# check_freg={}

# for data in dataDF['상품명']:
#     if data not in check_freg:
#         check_freg[data] = 1
#     else:
#         check_freg[data] += 1

In [17]:
# check_freg2={}

# for data in dataDF['상품(encoding)']:
#     if data not in check_freg2:
#         check_freg2[data] = 1
#     else:
#         check_freg2[data] += 1

In [21]:
check_freg1=checkEncoding('상품명')
check_freg2=checkEncoding('상품(encoding)')

In [24]:
checkDF=pd.DataFrame([check_freg1.values(), check_freg2.values()])
checkDF

,0,1,2,3,4,5,6,7,8,9,...,112,113,114,115,116,117,118,119,120,121
0,1040,1020,970,1000,1000,940,1000,1000,730,1000,...,890,890,890,880,880,880,880,880,880,880
1,1040,1020,970,1000,1000,940,1000,1000,730,1000,...,890,890,890,880,880,880,880,880,880,880


In [20]:
checkDF=checkDF.T

In [22]:
checkDF['T/F'] = checkDF[0] - checkDF[1]

In [23]:
checkDF['T/F'].sum()

0

In [24]:
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112010 entries, 0 to 112009
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0.1  112010 non-null  int64 
 1   상점            75665 non-null   object
 2   가격            112010 non-null  int64 
 3   상품명           112010 non-null  object
 4   시간            112010 non-null  object
 5   체크            112010 non-null  object
 6   어뷰징여부(0, 1)   112010 non-null  int64 
 7   상품(encoding)  112010 non-null  int32 
dtypes: int32(1), int64(3), object(4)
memory usage: 6.4+ MB


In [25]:
newDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112010 entries, 0 to 112009
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   상품명           112010 non-null  object
 1   상품(encoding)  112010 non-null  int32 
 2   어뷰징여부(0, 1)   112010 non-null  int64 
 3   시간            112010 non-null  object
 4   가격            112010 non-null  int64 
dtypes: int32(1), int64(2), object(2)
memory usage: 3.8+ MB


In [26]:
featureDF = newDF.drop(columns=['상품명','어뷰징여부(0, 1)','시간']) # 상품(encoding), 가격
targetDF = newDF[['어뷰징여부(0, 1)']] # 의심됨: 0, 정상가: 1

- 데이터셋 정의

In [27]:
# ---------------------------------------------------------------------
# 모델 이름 : CheatDataset
# 부모클래스 : Dataset 
# 매개 변수 : 
# ---------------------------------------------------------------------
class CheatDataset(Dataset):

    def __init__(self, featureDF, targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]

    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index])
        targetTS = torch.FloatTensor(self.targetDF.iloc[index])

        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [28]:
# - 커스텀데이터셋 인스턴스 생성
dataDS=CheatDataset(featureDF, targetDF)

- 모델 정의

In [39]:
# ---------------------------------------------------------------------
# 모델 이름 : MyCheatModel
# 부모클래스 : nn.Module 
# 매개 변수 : in_in, in_out, out_out, *hid
# ---------------------------------------------------------------------
class MyCheatModel(nn.Module):

    # 모델 구조 설계 함수 즉, 생성자 메서드
    def __init__(self, in_in, in_out, out_out, *hid):
        super().__init__()

        self.in_layer=nn.Linear(in_in, hid[0] if len(hid) else in_out)
        self.in_hids=nn.ModuleList()
        for i in range(len(hid)-1):
            self.in_hids.append(nn.Linear(hid[i], hid[i+1]))

        self.out_layer=nn.Linear(hid[-1] if len(hid) else in_out, out_out)

    # 학습 진행 콜백 메서드
    def forward(self, x):
        # 입력층
        y = F.relu(self.in_layer(x))

        # 은닉층
        for in_hid in self.in_hids:
            y = F.relu(in_hid(y))

        # 출력층
        return F.sigmoid(self.out_layer(y))

In [40]:
# 모델 인스턴스 생성
model = MyCheatModel(2, 250, 1, 250, 500, 250)
print(model)

MyCheatModel(
  (in_layer): Linear(in_features=2, out_features=250, bias=True)
  (in_hids): ModuleList(
    (0): Linear(in_features=250, out_features=500, bias=True)
    (1): Linear(in_features=500, out_features=250, bias=True)
  )
  (out_layer): Linear(in_features=250, out_features=1, bias=True)
)


- 하이퍼 파라미터 설정

In [41]:
# 학습 준비
# 하이퍼 파라미터 설정
EPOCHS = 100
BATCH_SIZE = 64
LR = 0.001

BATCH_CNT = dataDF.shape[0]//BATCH_SIZE

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'BATCH_CNT: {BATCH_CNT}')

BATCH_CNT: 1750


In [42]:
# 데이터셋 인스턴스 생성
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1, train_size=0.8, stratify=targetDF)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1, train_size=0.8, stratify=y_train)
print(f'[X_train(shape): {X_train.shape} (type): {type(X_train)}], X_test: {X_test.shape}, X_val: {X_val.shape}')
print(f'[y_train(shape): {y_train.shape} (type): {type(y_train)}], y_test: {y_test.shape}, y_val: {y_val.shape}')

[X_train(shape): (71686, 2) (type): <class 'pandas.core.frame.DataFrame'>], X_test: (22402, 2), X_val: (17922, 2)
[y_train(shape): (71686, 1) (type): <class 'pandas.core.frame.DataFrame'>], y_test: (22402, 1), y_val: (17922, 1)


- 데이터 스케일링 진행

In [43]:
# 학습용(trainDS) 기반 스케일러 생성
ss = StandardScaler()
SCALE=ss.fit(X_train)

# X_train_fit 스케일링 파일 저장 -> 모델 결과 확인용
with open('SCALE.pkl', 'wb') as f:
    pickle.dump(SCALE, f)

# 스케일링 진행
X_train_scaled = ss.transform(X_train)
X_val_scaled = ss.transform(X_val)
X_test_scaled = ss.transform(X_test)

In [44]:
# trainDS, valDS, testDS 인스턴스 생성
trainDS=CheatDataset(pd.DataFrame(X_train_scaled), y_train)
valDS=CheatDataset(pd.DataFrame(X_val_scaled), y_val)
testDS=CheatDataset(pd.DataFrame(X_test_scaled), y_test)

# 데이터로더 인스턴스 생성
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE, shuffle=True)
valDL=DataLoader(valDS, batch_size=BATCH_SIZE, shuffle=False)
testDL=DataLoader(testDS, batch_size=BATCH_SIZE, shuffle=False)

In [45]:
# 최적화 인스턴스 생성
optimizer = optim.Adam(model.parameters(), lr=LR)

# 최적화 스케쥴링 인스턴스 생성 => lr 조절 및 성능 개선 여부 체크
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=5, verbose=True) # score 사용 시
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True) # Loss 사용 시

# 손실함수 인스턴스 생성
loss_func = nn.BCELoss()

# 성능평가 함수
score_func = BinaryF1Score()

c:\Users\KDP-50\anaconda3\envs\WEB_AI\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


- training, validate 함수 정의

In [46]:
# ---------------------------------------------------------------------
# 함수 이름 : training
# 함수 역할 : 배치 크기 만큼 데이터 로딩해서 학습 진행
# 매개 변수 : 
# ---------------------------------------------------------------------

def training():
    # 학습 모드로 모델 설정
    model.train()
    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total=0,0
    for featureTS, targetTS in trainDL:

        # 학습 진행
        pre_y=model(featureTS)

        # 손실 계산
        loss=loss_func(pre_y, targetTS)
        loss_total+=loss.item()
        
        # 성능평가 계산
        score=score_func(pre_y, targetTS)
        score_total+=score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss_total, score_total

# ---------------------------------------------------------------------
# 함수 이름 : validate
# 함수 역할 : 배치 크기 만큼 데이터 로딩해서 검증 진행
# 매개 변수 : 
# ---------------------------------------------------------------------

def validate():
    # 검증 모드로 모델 설정
    model.eval()
    loss_val_total, score_val_total=0,0
    with torch.no_grad():
        # 검증 데이터셋
        for val_featureTS, val_targetTS in valDL:
            # val_featureTS=torch.FloatTensor(valDS.featureDF.values)
            # val_targetTS=torch.FloatTensor(valDS.targetDF.values)
        
            # 평가
            pre_val=model(val_featureTS)

            # 손실
            loss_val=loss_func(pre_val, val_targetTS)
            loss_val_total+=loss_val.item()

            # 성능평가
            score_val=score_func(pre_val, val_targetTS)
            score_val_total+=score_val.item()

    return loss_val_total, score_val_total

In [47]:
# 저장 경로
SAVE_PATH='../Final_project/CheatModels/'
funcMake.checkPath(SAVE_PATH)

경로있음


- 학습 진행

In [50]:
loss_history, score_history=[[],[]], [[],[]]
print('TRAIN, VAL 진행')

# => 임계기준 : 10번
BREAK_CNT = 0
THESHOLD = 100

for epoch in range(1, EPOCHS):
    # 학습 모드 함수 호출
    loss_total, score_total = training()

    # 검증 모드 함수 호출
    loss_val_total, score_val_total = validate()

    # 에포크당 손실값과 성능평가값 저장
    loss_history[0].append(loss_total/len(trainDL))
    score_history[0].append(score_total/len(trainDL))

    loss_history[1].append(loss_val_total/len(valDL))
    score_history[1].append(score_val_total/len(valDL))
    
    # Loss 기준
    if len(loss_history[1]) >= 2:
        if loss_history[1][-1] >= loss_history[1][-2]: BREAK_CNT += 1
    
    # 성능이 좋은 학습 가중치 저장 (validate 기준)
    # SAVE_MODEL=f'{epoch}_loss({loss_total/len(trainDL):.5f})_score({score_total/len(trainDL):.5f}).pth'
    SAVE_MODEL=f'{epoch}_loss({loss_val_total/len(valDL):.5f})_score({score_val_total/len(valDL):.5f}).pth'

    if len(score_history[1]) == 1:
        # 모델 전체 저장
        torch.save(model, SAVE_PATH+SAVE_MODEL)

    else:
        if score_history[1][-1] > max(score_history[1][:-1]):
            torch.save(model, SAVE_PATH+SAVE_MODEL)

    # 학습 중단 여부 설정
    if BREAK_CNT >= THESHOLD:
        print('성능 및 손실 개선이 없어서 학습 중단')
        break

    print(f'{epoch}/{EPOCHS} => [TRAIN] LOSS: {loss_history[0][-1]} SCORE: {score_history[0][-1]}')
    print(f'\t=> [VAL] LOSS: {loss_history[1][-1]} SCORE: {score_history[1][-1]}')

    # -----------------------------------------------------------------------------------------
    # # 최적화 스케쥴러 인스턴스 업데이트
    # scheduler.step(loss_val_total)
    # print(f'scheduler.num_bad_epochs => {scheduler.num_bad_epochs}', end=' ')
    # print(f'scheduler.patience => {scheduler.patience}')

    # # 손실 감소(또는 성능 개선)이 안되는 경우 조기종료
    # if scheduler.num_bad_epochs >= scheduler.patience:
    #     print(f'{scheduler.patience} EPOCH 성능 개선이 없어서 조기종료함')
    #     break
    # -----------------------------------------------------------------------------------------

    # 테스트 진행
    model.eval()
    with torch.no_grad():
        # 테스트 데이터셋
        test_featureTS=torch.FloatTensor(testDS.featureDF.values)
        test_targetTS=torch.FloatTensor(testDS.targetDF.values)
        # 평가
        pre_test=model(test_featureTS)

        # 손실
        loss_test=loss_func(pre_test, test_targetTS)

        # 성능평가
        score_test=score_func(pre_test, test_targetTS)
    print(f'\t=>[TEST] LOSS: {loss_test} SCORE: {score_test}')

TRAIN, VAL 진행
1/100 => [TRAIN] LOSS: 0.31827024081314814 SCORE: 0.8850576136922539
	=> [VAL] LOSS: 0.34008092637376003 SCORE: 0.8826728732560453
	=>[TEST] LOSS: 0.33566001057624817 SCORE: 0.8861851096153259
2/100 => [TRAIN] LOSS: 0.3217855281429988 SCORE: 0.879931749987028
	=> [VAL] LOSS: 0.3099566522017917 SCORE: 0.884022416381225
	=>[TEST] LOSS: 0.30567634105682373 SCORE: 0.8872107267379761
3/100 => [TRAIN] LOSS: 0.31925755653141547 SCORE: 0.8824211770567268
	=> [VAL] LOSS: 0.3257707352103712 SCORE: 0.8772328209622475
	=>[TEST] LOSS: 0.31768321990966797 SCORE: 0.8820483088493347
4/100 => [TRAIN] LOSS: 0.31762138851831045 SCORE: 0.8836145211180654
	=> [VAL] LOSS: 0.3143178778717102 SCORE: 0.8809549396996821
	=>[TEST] LOSS: 0.30781877040863037 SCORE: 0.8862980008125305
5/100 => [TRAIN] LOSS: 0.3166667980795774 SCORE: 0.8843425222532117
	=> [VAL] LOSS: 0.3200876024482089 SCORE: 0.8894277758021372
	=>[TEST] LOSS: 0.3117420971393585 SCORE: 0.8939501047134399
6/100 => [TRAIN] LOSS: 0.31373

In [51]:
featureDF.to_csv('feature.csv')
targetDF.to_csv('target.csv')